# Get pool shapes

This little notebook's purpose is to extract pool/park shapes from the City of San Antonio's shapefile. I got them from [here](https://www.sanantonio.gov/GIS/GISData#P). I also used [mapshaper](https://mapshaper.org/) to convert the shapefile to GeoJSON.

In [44]:
import os

import geopandas as gpd
import pandas as pd

In [45]:
geo_df = gpd.read_file('../data/ParkBoundaries.json', driver='GeoJSON')

# Set the crs to 'EPSG:2278'
geo_df['geometry'].crs = 'EPSG:2278'

geo_df.head()

,ParkID,ParkName,AltParkNam,StreetNumb,StreetName,ParkType,Shape_Leng,Shape_Area,geometry
0,19,Brackenridge Park,,0,,,27445.605050,1.109048e+07,"MULTIPOLYGON (((2138150.140 13716434.511, 2138..."
1,174,Oakhaven Park,,0,,,3316.874678,4.457859e+05,"POLYGON ((2139950.280 13763578.460, 2139950.89..."
2,353,Huesta Creek Greenway,,0,,,7386.307607,1.573662e+06,"POLYGON ((2085383.294 13755671.764, 2085033.31..."
3,346,Timber Ridge Park,,0,,,1550.700179,1.454740e+05,"POLYGON ((2078775.117 13718175.114, 2078760.42..."
4,446,Stone Oak Elementary School Park,,0,,,3657.784504,2.933213e+05,"MULTIPOLYGON (((2126726.040 13779299.166, 2126..."


In [46]:
pools_of_interest = ['san pedro', 'woodlawn', 'lions', 'lady bird', 'roosevelt', 'heritage', 'concepcion', 'elmendorf', 'dellview', 'normoyle']

# Filter the geo_df to only only include rows with a ParkName that contains one of the strings in pools_of_interest. 
# Hint: use the .str.contains() method on the ParkName column
# Hint: Lowercase the ParkName column before using .str.contains()
# Hint: Use the .reset_index() method to reset the index of the dataframe

# Lowercase the ParkName column
geo_df['ParkName'] = geo_df['ParkName'].str.lower()

geo_df = geo_df[geo_df['ParkName'].str.contains('|'.join(pools_of_interest))]

# Convert the geometry column to a crs of EPSG:4326
geo_df = geo_df.to_crs(epsg=4326)

# Order the dataframe by the ParkName column
geo_df = geo_df.sort_values(by='ParkName')

print(geo_df['geometry'].crs)

geo_df

EPSG:4326


,ParkID,ParkName,AltParkNam,StreetNumb,StreetName,ParkType,Shape_Leng,Shape_Area,geometry
289,47,concepcion park,,0,,,4301.606805,8.959870e+05,"POLYGON ((-98.49634 29.39035, -98.49644 29.390..."
55,419,dellview elementary school park,,0,,,2239.140015,1.656268e+05,"POLYGON ((-98.54485 29.50178, -98.54492 29.501..."
120,59,dellview park,,0,,,2594.325522,2.194910e+05,"POLYGON ((-98.53970 29.50452, -98.53970 29.504..."
232,64,elmendorf lake park,,0,,,6637.653489,1.614197e+06,"POLYGON ((-98.54262 29.42826, -98.54275 29.427..."
300,93,heritage duck pond park,,0,,,2156.323470,2.942246e+05,"POLYGON ((-98.68223 29.42787, -98.68222 29.427..."
370,94,heritage pool park,,0,,,1301.456246,1.022075e+05,"POLYGON ((-98.67957 29.42013, -98.67960 29.420..."
21,119,lady bird johnson park,,0,,,7622.207115,3.365808e+06,"POLYGON ((-98.42378 29.53281, -98.42114 29.535..."
35,156,mission concepcion park,,0,,,1993.080613,1.132751e+05,"POLYGON ((-98.49091 29.39082, -98.49091 29.390..."
174,169,normoyle park,,0,,,5281.749418,1.351039e+06,"POLYGON ((-98.53897 29.37852, -98.53882 29.379..."
133,208,roosevelt park,,0,,,3225.584504,3.573852e+05,"MULTIPOLYGON (((-98.48646 29.40024, -98.48646 ..."


In [47]:
# For each pool, export the shape of the pool to a GeoJSON file
# Hint: Use the .to_file() method on the geodataframe
# Hint: Use the .ParkName column to name the file

for index, row in geo_df.iterrows():
    # Replace spaces with underscores
    park_name = row['ParkName'].replace(' ', '_')

    geo_df[geo_df['ParkName'] == row['ParkName']].to_file(f'../output/shapes/{park_name}.json', driver='GeoJSON')